In [ ]:
#코드들 임의로 합침

In [2]:
!pip install newsapi-python
!pip install requests beautifulsoup4
!pip install sumy
!pip install konlpy
!pip install nltk scikit-learn

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 21.9 MB/s eta 0:00:00
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21691 sha256=043a586d25b04e282110c334c634b5500afb19d986260006a194ea660fff45ac
  Stored in directory: /root/.cache/pip/wheels/64/22/90/b84fcc30e16598db20a0d41340616dbf9b1e82bbcc627b0b33
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13704 sha256=3474056a26817e2db087cd98b8233787bed44d05e7f0c4e9a4a0dab9b8dad392
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built breadability docopt
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 26.0 MB/s eta 0:00:00


In [3]:
import requests
from bs4 import BeautifulSoup
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import re

# 네이버 API 키 설정
client_id = 'xIpH54vcwURuvzz8t0vl'
client_secret = 'dfHi2SP3CQ'

# API 요청 URL
url = 'https://openapi.naver.com/v1/search/news.json'

# 카테고리 설정
categories = {
    '경제': '경제',
    'IT': 'IT',
    '과학': '과학',
    '사회': '사회',
    '문화': '문화',
    '스포츠': '스포츠',
    '건강': '건강'
}

# 사용자로부터 카테고리 선택 받기
print("카테고리를 선택하세요: 경제, IT, 과학, 사회, 문화, 스포츠, 건강")
user_choice = input("선택한 카테고리: ")

# 유효한 카테고리인지 확인
if user_choice in categories:
    selected_category = categories[user_choice]
else:
    print("유효하지 않은 카테고리입니다. 기본적으로 '경제' 카테고리를 선택합니다.")
    selected_category = '경제'

# 파라미터 설정
params = {
    'query': selected_category,  # 검색어
    'display': 5,            # 5개의 뉴스 가져오기
    'sort': 'date'           # 날짜 기준 정렬
}

# 헤더 설정
headers = {
    'X-Naver-Client-Id': client_id,
    'X-Naver-Client-Secret': client_secret
}

# 기사 내용 추출 함수
def extract_text_from_url(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Failed to retrieve the web page. Status code: {response.status_code}")
            return ""

        soup = BeautifulSoup(response.content, 'html.parser')
        for script_or_style in soup(['script', 'style']):
            script_or_style.decompose()

        text = soup.get_text(separator=' ')
        text = ' '.join(text.split())

        return text

    except requests.exceptions.RequestException as e:
        print(f"Error during requests to {url}: {str(e)}")
        return ""

# API 요청
response = requests.get(url, params=params, headers=headers)

# 응답을 JSON 형태로 파싱
data = response.json()

# 뉴스 정보 출력 및 기사 내용 추출
if 'items' in data:
    articles = data['items']
    for i, article in enumerate(articles, 1):
        title = article['title']
        description = article['description']
        link = article['link']
        pub_date = article['pubDate']

        print(f"뉴스 {i}:")
        print(f"제목: {title}")
        print(f"URL: {link}")
        print(f"발행일: {pub_date}")

        # URL에서 기사 텍스트 추출
        extracted_text = extract_text_from_url(link)

        if extracted_text:
            # 요약 기능
            parser = PlaintextParser.from_string(extracted_text, Tokenizer("korean"))
            summarizer = TextRankSummarizer()
            summary = summarizer(parser.document, 3)
            summary_text = " ".join(str(sentence) for sentence in summary)

            # 요약된 텍스트가 3000자를 넘지 않도록 자르기
            if len(summary_text) > 3000:
                summary_text = summary_text[:3000] + '...'

            print("내용 요약:")
            print(summary_text)

            # 키워드 추출 기능
            okt = Okt()

            def extract_keywords(text, num_keywords=5):
                text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\\s]', '', text)
                nouns = okt.nouns(text)
                if not nouns:
                    return []
                vectorizer = TfidfVectorizer(max_features=num_keywords)
                tfidf_matrix = vectorizer.fit_transform([' '.join(nouns)])
                feature_names = vectorizer.get_feature_names_out()
                sorted_items = sorted(zip(vectorizer.idf_, feature_names))
                keywords = [item[1] for item in sorted_items[:num_keywords]]
                return keywords

            keywords = extract_keywords(summary_text, num_keywords=5)

            print(f"키워드: {keywords}")
        else:
            print("기사를 추출할 수 없습니다.")
        print("-" * 80)
else:
    print("뉴스를 가져오는데 실패했습니다.")


카테고리를 선택하세요: 경제, IT, 과학, 사회, 문화, 스포츠, 건강
선택한 카테고리: 경제
뉴스 1:
제목: 삼성·GM 배터리 공장 설립…인디애나주에 35억불 투자
URL: https://news.koreadaily.com/2024/08/27/society/generalsociety/20240827205630110.html
발행일: Wed, 28 Aug 2024 12:58:00 +0900
내용 요약:
지역 홈 LA 중앙일보 뉴욕 중앙일보 애 틀 랜 타 중앙일보 시카고 중앙일보 워싱턴 중앙일보 달라 스중앙 일보 덴버 중앙일보 샌 디에 고 중앙일보 밴쿠버 중앙일보 토론토 중앙일보 한국 중앙일보 많이 본 뉴스 [Los Angeles] 입력 + A - A 기사 공유 삼성 ·GM 배터리 공장 설립… 인 디 애나 주에 35 억불 투자 삼성 SDI가 제너럴 모터스( 이하 GM) 와 손잡고 미국 내 전 기차 배터리 생산 공장을 짓기로 확정했다. 삼성 SDI는 GM 과의 합작법인을 통해 북미 전 기차 시장에서 점유율을 확대하기 위한 발판을 마련하고, 각 형 배터리 채용 고객 사를 늘려 나갈 계획이다. 최 윤 호 사장은 " 북미 지역 1위 완성 차업체인 GM과 지난해 MOU 체결 이후 굳건한 파트너십을 바탕으로 미국 전 기차 시장을 선도할 프리미엄 배터리 생산 거점을 마련하게 됐다 "며 " 삼성 SDI 만의 초 격차 기술력을 담은 프라이 맥스 (PRiMX) 배터리 제품으로 GM이 전기 차 시장 리더십을 강화하는 데 최선의 노력을 다하겠다" 고 말했다.
키워드: ['배터리', '삼성', '일보', '전기차', '중앙']
--------------------------------------------------------------------------------
뉴스 2:
제목: 삼성SDI-GM, 배터리 합작법인 설립... 2027년 양산 목표
URL: https://www.megaeconomy.co.kr/news/newsview.php?ncode=10655848944570